In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor

import harness

import timeit

np.warnings.filterwarnings('ignore', 'Ill-conditioned matrix')
pd.options.display.max_columns=None

In [2]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [3]:
flights_clean = harness.clean_train(flights_train)


add_features = harness.chain(
    harness.add_date_parts, 
    harness.add_haul, 
    harness.add_weather,
    harness.add_hour,
#    harness.make_all_dummies
)


flights_with_features = add_features(flights_clean)


x_transform = harness.chain(
    harness.keep_only_test_columns,
    add_features,
    [harness.add_all_grouped_stats, flights_with_features],
    harness.only_numeric,
    harness.scale,
)


transformer = harness.DataTransformer(
    x_transform, harness.normalize, harness.unnormalize
)


x_train, y_train = transformer.extract_transform(flights_clean)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PowerTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
for col in x_train.columns:
    print(f"{col}: {np.corrcoef(x_train[col], y_train.arr_delay)[0, 1]}")

In [ ]:
def select_columns(df):
    return df[[
        'crs_dep_time',
        'crs_arr_time',
        'crs_elapsed_time',
        'distance',
        'origin_city_name_delay_mean',
        'dest_city_name_delay_mean',
        'origin_airport_id_delay_mean',
        'dest_airport_id_delay_mean',
        'day_delay_mean',
        'month_delay_mean',
        'op_unique_carrier_delay_mean',
        'haul_delay_mean',
        'tail_num_delay_mean',
        'hour_delay_mean',
    ]]





x_transform = harness.chain(x_transform, select_columns)
transformer = harness.DataTransformer(
    x_transform
)
x_train, y_train = transformer.extract_transform(flights_clean)

In [7]:
def train_model():
    GBR = GradientBoostingRegressor(n_estimators=10)
    grid_search = ms.GridSearchCV(
        GBR, {'max_depth' : [None, 2,3,5],'max_leaf_nodes' : [None, 3,5,10]}
    )
    grid_search.fit(x_train.values, y_train.values)
    harness.save(grid_search.best_estimator_, 'GradientBoosting_model')

In [8]:
train_model()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

In [9]:
GBR = harness.load('GradientBoosting_model')
GBR_model = harness.TrainedModel(GBR, transformer)

GBR_model.validate(harness.clean_train(flights_test))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PowerTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


R squared: -0.0529
Median absolute error: 10

In [ ]:
GBR_model.submit(
    'data/test.csv', 'time_of_day_submission.csv', 'predicted_delay'
)